In [1]:
import urllib
import bs4
import re
import pandas as pd
import random

from time import sleep

In [2]:
url_landing = "https://leagueoflegends.fandom.com/wiki/List_of_champions/Base_statistics"
req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()
stats_champions_lol = bs4.BeautifulSoup(webpage, "lxml")

###

url_landing = "https://leagueoflegends.fandom.com/wiki/List_of_champions_in_Dawn_of_Heroes_(Teamfight_Tactics)"
req = urllib.request.Request(url_landing, headers={'User-Agent':'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()
list_champs_tft = bs4.BeautifulSoup(webpage, "lxml")

In [3]:
headers = []
for item in stats_champions_lol.findAll('th')[0:19]:
    #print(item.getText())
    headers.append(item.getText())
    
champions_stats = []

for item in stats_champions_lol.findAll('td'):
    #print(item.getText())
    champions_stats.append(item.getText())
    
n = len(headers)
lol_champ_w_stats = [champions_stats[i * n:(i + 1) * n] for i in range((len(champions_stats) + n - 1) // n )]
lol_champ_w_stats = lol_champ_w_stats[:-1]

df_lol_champs = pd.DataFrame(lol_champ_w_stats, columns = headers)

In [4]:
df_lol_champs['Champions'][130]

' Tryndamere'

In [5]:
#get headers

url_champ = "https://leagueoflegends.fandom.com/wiki/Aatrox/LoL"
req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")

headers_stats = ['Title']
spells = ['Passive','Q Spell','W Spell','E Spell','Ultimate']
kda = ['KDA_kills','KDA_deaths','KDA_assists']

for item in page_champ.findAll('h3',{'class':'pi-data-label pi-secondary-font'})[:11]:
    headers_stats.append(item.getText())

headers_stats = headers_stats + spells

url_champ = "https://www.leagueofgraphs.com/champions/stats/ezreal"
req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")

for item in page_champ.findAll('div',{'class':'pie-chart-title pie-chart-title-bottom'}):
    headers_stats.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))
    
for item in page_champ.findAll('div',{'class':'title'})[2:]:
    headers_stats.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))

headers_stats = headers_stats + kda

#init
i = 1
items_champ = []

#get data

for champ in df_lol_champs['Champions']:       
    
    #get data from fandom
    url_champ = "https://leagueoflegends.fandom.com/wiki/"+champ.replace(' ','_')+"/LoL"
    req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
    page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")
    
    for item in page_champ.findAll('div',{'class':'pi-data-value pi-font'})[:12]:
        items_champ.append(item.getText())
        
    for item in page_champ.findAll('span',{'class':'mw-headline'})[1:6]:
        items_champ.append(item.getText())
    
    
    #get data from league of graphs
    
    url_champ = "https://www.leagueofgraphs.com/champions/stats/"+champ.replace(' ','').replace("'","").replace('.','').replace('Mega','').replace('Wukong','monkeyking').lower()
    req = urllib.request.Request(url_champ, headers={'User-Agent':'Mozilla/5.0'})
    page_champ = bs4.BeautifulSoup(urllib.request.urlopen(req).read(), "lxml")
    
    for item in page_champ.findAll('div',{'class':'pie-chart'}):
        items_champ.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))
    
    for item in page_champ.findAll('div',{'class':'number solo-number'})[1:]:
        items_champ.append(item.getText().replace(' ','').replace('\n','').replace('\r',''))

    for item in page_champ.findAll('span',{'class':'kills'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    for item in page_champ.findAll('span',{'class':'deaths'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    for item in page_champ.findAll('span',{'class':'assists'})[0]:
        items_champ.append(item.replace(' ','').replace('\n','').replace('\r',''))
    
            
    print(str(i)+" / "+str(len(df_lol_champs['Champions']))+" champions importés")
    
    i+=1
    
    sleep(random.randint(5,10))

###    
    


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)>

In [81]:
n = len(headers_stats)
l_test = [items_champ[i * n:(i + 1) * n] for i in range((len(items_champ) + n - 1) // n )]

test = pd.DataFrame(l_test, columns = headers_stats)

In [82]:
df_lol_champs = pd.concat([df_lol_champs, test.reindex(df_lol_champs.index)], axis=1)

In [84]:
df_lol_champs.tail()

,Champions,HP,HP+,HP5,HP5+,MP,MP+,MP5,MP5+,AD,...,BanRate,Mainedby,PentaKill,Gold,Minions,Wards,DamageDealt,KDA_kills,KDA_deaths,KDA_assists
153,Zed,584,+85,7,+0.65,200,+0,50,+0,63,...,42.7%,1.7%,0.0014,"11,846",169.0,7.5,"21,236",8.7,6.3,5.4
154,Ziggs,536,+92,6.5,+0.6,480,+23.5,8,+0.8,54,...,8.2%,0.4%,0.0007,"11,508",186.5,9.6,"24,017",5.4,5.8,8.5
155,Zilean,504,+82,5.5,+0.5,452,+50,11.34,+0.8,52,...,1.1%,0.6%,0.0000,"8,400",50.3,22.2,"10,639",2.5,5.0,13.3
156,Zoe,560,+92,6.5,+0.6,425,+25,8,+0.65,58,...,1.0%,0.3%,0.0007,"10,345",147.6,9.9,"19,702",6.3,5.7,7.2
157,Zyra,504,+79,5.5,+0.5,418,+25,13,+0.4,53,...,2.2%,0.8%,0.0002,"9,397",53.4,25.6,"20,102",4.0,7.1,10.8


In [85]:
df_lol_champs.to_csv('lol_champs.csv')